# Transformando PDF para Dataframe (Brumadinho)
Neste projeto é aplicado técnicas de OCR para transformar o PDF disponibilizado pela vale em um dataframe do pandas, que também pode ser salvo em CSV.


In [1]:
# instalando requisitos
# !apt update -y && apt upgrade -y
# !apt install -y imagemagick tesseract-ocr libtesseract libtesseract-dev
!pip install -r requirements.txt

In [182]:
# Importando bibliotacas
import re
import csv
import os
import pandas as pd
import numpy as np
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import matplotlib as mpl
from matplotlib import pyplot as plt
from os import walk
from natsort import natsorted, ns
from PIL import Image
import PIL.ImageOps  
import pytesseract
import cv2
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

plt.rcParams['figure.dpi'] = 300
mpl.rcParams['figure.figsize'] = (15,15)

%config IPCompleter.greedy=True

In [179]:
# função para definir onde imagem deve ser cortada
def get_crop_size(path):
    page_img = Image.open(path)

    top = 0
    left = 0
    right = page_img.size[0]
    bottom = page_img.size[1]
    
    #finding top
    for i in range(0,500):
        pixel = page_img.getpixel((i,i))
        pixel_mean = np.mean(pixel)
        if pixel_mean < 200 and top == 0:
            top = i

    # finding left
    for i in range(0,500):
        pixel = page_img.getpixel((i,top))
        pixel_mean = np.mean(pixel)
        if pixel_mean < 240 and left == 0:
            left = i

    return (left,top + 220,left + 1565,top + 3000)

# função para recuperar dados da página
# fonte: https://github.com/dieegom/brumadinho_location/blob/master/crawler/crawler.py

def get_raw_data():
    url = "http://brumadinho.vale.com/listagem-pessoas-sem-contato.html"
    r = requests.get(url)
    r.encoding = 'utf-8'
    return r.content

In [42]:
# Definindo cconstantes

categoryList = ['Terceiro/Comunidade','Próprios']
statusList = ['Localizado','Sem contato','Óbito confirmado pelo IML', 'Alta']

doc_upper = 0
doc_lower = 3260

path = "pages"
# path = "/tmp/pages"

In [6]:
# Recuperando endereço do PDF a partir da página a vale
html = get_raw_data()
soup = BeautifulSoup(html, 'html.parser')
pdf = ''
for a in soup.find_all('a', href=True):
    href = a['href']
    if href.endswith('pdf'):
        pdf = href

filename = pdf.split('/')[-1]

In [7]:
# Convertendo PDF para PNG
# é importante comentar a linha <policy domain="coder" rights="none" pattern="PDF" /> em /etc/ImageMagick-6/policy.xml

!mkdir $path
!convert -density 300 $pdf $path/page.png

mkdir: cannot create directory ‘pages’: File exists


In [22]:
# Criando lista de páginas

f = []
for (dirpath, dirnames, filenames) in walk(path):
    f.extend(filenames)
    break

pages = natsorted(f, key=lambda y: y.lower())
pages

['page-0.png',
 'page-1.png',
 'page-2.png',
 'page-3.png',
 'page-4.png',
 'page-5.png',
 'page-6.png',
 'page-7.png',
 'page-8.png',
 'page-9.png',
 'page-10.png',
 'page-11.png']

In [180]:
# Criando DataSet

dataset = {
    'nomes':[],
    'categorias':[],
    'status':[]
}

for page in pages:
    crop = get_crop_size("{}/{}".format(path,page))
    page_img = Image.open("{}/{}".format(path,page))
    page_img = page_img.crop(crop)
    
    segments = {
        'nomes':      (0,0,775,page_img.size[1]),
        'categorias': (775,0,1150,page_img.size[1]),
        'status':     (1150,0,page_img.size[0],page_img.size[1]),
    }

#     page_img = Image.open("{}/{}".format(path,page))
    for seg_name in segments.keys():
        image = page_img.crop(segments[seg_name])
        
        image_text = pytesseract.image_to_string(image)
        image_text_list = image_text.split('\n')
        image_text_list = list(filter(lambda x: len(x.strip()) > 0, image_text_list))
        
        if seg_name != 'topo':
            dataset[seg_name] += image_text_list
            print(page, seg_name, len(image_text_list))

page-0.png nomes 67
page-0.png categorias 67
page-0.png status 67
page-1.png nomes 67
page-1.png categorias 67
page-1.png status 67
page-2.png nomes 67
page-2.png categorias 67
page-2.png status 67
page-3.png nomes 67
page-3.png categorias 67
page-3.png status 67
page-4.png nomes 67
page-4.png categorias 67
page-4.png status 67
page-5.png nomes 67
page-5.png categorias 67
page-5.png status 67
page-6.png nomes 67
page-6.png categorias 67
page-6.png status 67
page-7.png nomes 67
page-7.png categorias 67
page-7.png status 67
page-8.png nomes 41
page-8.png categorias 41
page-8.png status 41
page-9.png nomes 38
page-9.png categorias 38
page-9.png status 38
page-10.png nomes 48
page-10.png categorias 48
page-10.png status 48
page-11.png nomes 46
page-11.png categorias 46
page-11.png status 46


## Legenda

**categorias_2**

1. Terceiro/Comunidade
2. Próprios

**status_2**

1. Localizado
2. Sem contato
3. Óbito confirmado pelo IML
4. Alta

In [181]:
# Criando DataFrame

df = pd.DataFrame(dataset)
df['categorias_2'] = [categoryList.index(process.extractOne(test_word, categoryList)[0]) +1 for test_word in df['categorias'].values]
df['status_2'] = [ statusList.index(process.extractOne(test_word, statusList)[0]) + 1 for test_word in df['status'].values]
df.to_csv(filename.replace('pdf','csv'),index = False)
df

,nomes,categorias,status,categorias_2,status_2
0,ABNER PEREIRA DOS SANTOS,Prdéprios,Localizado,2,1
1,ADAIL DOS SANTOS JUNIOR,Terceiro/Comunidade,Sem Contato,1,2
2,ADAIR CUSTODIO RODRIGUES,Préprios,Sem Contato,2,2
3,ADAIR MARTINS,Préprios,Localizado,2,1
4,ADEILSON HENRIQUE GOMES RABELO,Terceiro/Comunidade,Localizado,1,1
5,ADELSON CLAUDIO DE JESUS,Prdéprios,Localizado,2,1
6,ADEMARIO BISPO,Terceiro/Comunidade,Sem Contato,1,2
7,ADILSON DOS SANTOS (EMPRESA TERCEIRIZADA/COMUN...,Terceiro/Comunidade,Localizado,1,1
8,ADILSON DOS SANTOS (VALE),Prdéprios,Localizado,2,1
9,ADILSON SATURNINO DE SOUZA,Terceiro/Comunidade,Sem Contato,1,2
